In [5]:
"""setup"""
import json
from spotifydb import *
from IPython.display import display
import matplotlib.pyplot as plt
import os

In [ ]:
"""Create tables"""

try: os.remove("spotify.db")
except FileNotFoundError: pass
connection = connect("spotify.db")
cursor = connection.cursor()
with open("database.sql", 'r') as f:
    cursor.executescript(f.read())
connection.commit()
connection.close()

In [3]:
"""populate database"""

data_directory = "data/elias"

# from os import listdir
# from os.path import isfile, join
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

connection = connect("spotify.db")
for filename in os.listdir(data_directory):
    with open(f"{os.path.join(data_directory, filename)}", 'r', encoding='utf8') as f:
        streams = json.load(f)
    for stream in streams:
        if not add_song_stream(connection, stream):
            add_episode_stream(connection, stream)
connection.commit()
connection.close()

In [ ]:
"""return all songs ranked by times played"""

query = """--sql
    SELECT so.song_name as Song, COUNT(so.url) AS "Times played"
    FROM songs so JOIN song_streams st ON so.url = st.url
    GROUP BY so.song_name ORDER BY "Times played" DESC;
"""
result = run_single_query(query)
display(result)

In [ ]:
"""returns all albums ranked by times played"""
query = """--sql
    SELECT so.album_name as Album, so.artist_name AS Artist, COUNT(LOWER(so.album_name)) AS "Times Played"
    FROM songs so JOIN song_streams st ON so.url = st.url
    GROUP BY LOWER(so.album_name)
    ORDER BY "Times Played" DESC
    LIMIT 40;
"""
result = run_single_query(query)
display(result)

In [ ]:
"""return all songs ranked by times played"""

query = """--sql
    SELECT so.song_name as Song, COUNT(so.url) AS "Times played"
    FROM songs so JOIN song_streams st ON so.url = st.url
    WHERE st.time_stamp >= '2018-01-01 00:00:00'
    GROUP BY so.url, so.song_name ORDER BY "Times played" DESC
    LIMIT 50;
"""
result = run_single_query(query)
display(result)

In [ ]:
query = """--sql
    SELECT so.song_name as Song, so.album_name as Album, COUNT(so.song_name) AS "Times played"
    FROM songs so JOIN song_streams st ON so.url = st.url
    WHERE st.time_stamp >= '2024-01-01 00:00:00'
    GROUP BY so.song_name ORDER BY "Times played" DESC
    LIMIT 50;
"""

result = run_single_query(query)
display(result)

In [ ]:
"""return all songs ranked by minutes played"""
query = """--sql
    SELECT so.artist_name as Artist, Sum(st.ms_listened) / 60000 AS "Minutes listened"
    FROM songs so JOIN song_streams st ON so.url = st.url
 
    GROUP BY so.artist_name ORDER BY "Minutes listened" DESC
    LIMIT 50;
"""
result = run_single_query(query)
display(result)

In [ ]:
query = """--sql
    SELECT Sum(st.ms_listened) / 60000 AS "Minutes"
    FROM song_streams st
    WHERE st.time_stamp >= '2024-01-01 00:00:00';
"""

result = run_single_query(query)
display(result)

In [ ]:
query = """--sql
    SELECT pe.podcast AS Podcast, Count(pe.podcast) AS "Times Listened"
    FROM podcast_episodes pe JOIN podcast_streams ps ON pe.url = ps.url
    WHERE ps.ms_listened >= 120000
    GROUP BY pe.podcast ORDER BY "Times Listened" DESC;
"""

result = run_single_query(query)
display(result)

In [ ]:
query = """--sql
    SELECT pe.podcast AS Podcast, Count(pe.episode) AS "Number of Episodes"
    FROM podcast_episodes pe
    GROUP BY pe.podcast ORDER BY "Number of Episodes" DESC;
"""

result = run_single_query(query)
display(result)

In [ ]:


query = """--sql
    SELECT strftime('%m', st.time_stamp) AS "Month of year", SUM(st.ms_listened) 
    FROM song_streams st
    WHERE st.time_stamp >= '2024-01-01 00:00:00'
    ORDER BY "Hour of day";
"""

result = run_single_query(query)
vals = [int(y) for y in result['Month of year']]
plt.hist(vals)